In [13]:
# !pip install elasticsearch pandas tqdm
# from elasticsearch import Elasticsearch
#
# # Adjust host/port to your cluster
# es = Elasticsearch("http://localhost:19200")
#
# # Test connection
# print(es.info())


In [27]:
import sqlite3

main_db_path = "export_data.sqlite"
mapping_db_path = "export_data_mapping.sqlite"

conn = sqlite3.connect(main_db_path)
cursor = conn.cursor()

cursor.execute("PRAGMA foreign_keys = ON;")

cursor.execute("CREATE TABLE IF NOT EXISTS persons (id TEXT PRIMARY KEY)")
cursor.execute("CREATE TABLE IF NOT EXISTS occurrences (id TEXT PRIMARY KEY)")
cursor.execute("CREATE TABLE IF NOT EXISTS manuscripts (id TEXT PRIMARY KEY)")
cursor.execute("""
CREATE TABLE IF NOT EXISTS verses (
    id TEXT PRIMARY KEY,
    text TEXT,
    occurrence_id TEXT,
    manuscript_id TEXT,
    order_in_occurrence INTEGER,
    FOREIGN KEY (occurrence_id) REFERENCES occurrences(id),
    FOREIGN KEY (manuscript_id) REFERENCES manuscripts(id)
)
""")
cursor.execute("CREATE TABLE IF NOT EXISTS bibliographies (id TEXT PRIMARY KEY)")
cursor.execute("CREATE TABLE IF NOT EXISTS types (id TEXT PRIMARY KEY, text_original TEXT)")

conn.commit()
conn.close()

print(f"Main DB: '{main_db_path}', Mapping DB: '{mapping_db_path}' created with separate tables.")

import sqlite3
import requests

es_url = "http://localhost:19200"
main_db_path = "export_data.sqlite"
mapping_db_path = "export_data_mapping.sqlite"

conn = sqlite3.connect(main_db_path)
cursor = conn.cursor()
cursor.execute("PRAGMA foreign_keys = ON;")
cursor.execute(f"ATTACH DATABASE '{mapping_db_path}' AS mapping;")

indices_resp = requests.get(f"{es_url}/_cat/indices?format=json")
indices_resp.raise_for_status()
all_indices = [idx["index"] for idx in indices_resp.json() if idx["index"].startswith("dbbe_dev")]

def flatten_properties(properties, parent=""):
    flat_fields = []
    for field, value in properties.items():
        full_name = f"{parent}.{field}" if parent else field
        if "properties" in value:
            flat_fields.extend(flatten_properties(value["properties"], full_name))
        if "fields" in value:
            flat_fields.extend(flatten_properties(value["fields"], full_name))
        if "type" in value:
            flat_fields.append(full_name)
    return flat_fields

for index_name in all_indices:
    mapping_resp = requests.get(f"{es_url}/{index_name}/_mapping")
    mapping_resp.raise_for_status()
    index_mapping = mapping_resp.json()[index_name]["mappings"]["properties"]

    table_name = f"mapping_{index_name}"
    cursor.execute(f"""
        CREATE TABLE IF NOT EXISTS mapping.{table_name} (
            field_name TEXT PRIMARY KEY,
            mapped_to TEXT
        )
    """)

    flat_fields = flatten_properties(index_mapping)
    for field_name in flat_fields:
        cursor.execute(f"""
            INSERT OR IGNORE INTO mapping.{table_name} (field_name)
            VALUES (?)
        """, (field_name,))

conn.commit()
conn.close()
print("Mapping tables created and all nested fields populated.")


OperationalError: near "1": syntax error

## Verses

In [15]:
from elasticsearch import Elasticsearch
import sqlite3

es = Elasticsearch("http://localhost:19200")

main_db_path = "export_data.sqlite"
mapping_db_path = "export_data_mapping.sqlite"

conn = sqlite3.connect(main_db_path)
cursor = conn.cursor()
cursor.execute("PRAGMA foreign_keys = ON;")
cursor.execute(f"ATTACH DATABASE '{mapping_db_path}' AS mapping;")

indices = es.cat.indices(format="json")
indices = [idx['index'] for idx in indices if idx['index'].startswith("dbbe_dev")]

cursor.execute("""
CREATE TABLE IF NOT EXISTS verse_groups (
    id TEXT PRIMARY KEY
)
""")

cursor.execute("PRAGMA table_info(verses)")
existing_cols = [row[1] for row in cursor.fetchall()]

if "verse_group_id" not in existing_cols:
    cursor.execute("""
        ALTER TABLE verses ADD COLUMN verse_group_id TEXT REFERENCES verse_groups(id)
    """)

def scroll_all(index, query={"query": {"match_all": {}}, "size": 1000}):
    resp = es.search(index=index, body=query, scroll='2m')
    scroll_id = resp['_scroll_id']
    hits = resp['hits']['hits']
    all_hits = hits[:]
    while len(hits):
        resp = es.scroll(scroll_id=scroll_id, scroll='2m')
        hits = resp['hits']['hits']
        all_hits.extend(hits)
    return all_hits

def get_table_columns(table_name):
    cursor.execute(f"PRAGMA table_info({table_name})")
    return [row[1] for row in cursor.fetchall()]

def map_field_to_column(field_name, table_name):
    if table_name == "verses":
        mapping = {
            "id": "id",
            "manuscript.id": "manuscript_id",
            "occurrence.id": "occurrence_id",
            "verse": "text",
            "order": "order_in_occurrence"
        }
        return mapping.get(field_name)
    if table_name == "manuscripts":
        return "id" if field_name == "id" else None
    if table_name == "occurrences":
        return "id" if field_name == "id" else None
    return None

def mark_fields_migrated(index_name, table_name):
    cursor.execute(f"SELECT field_name FROM mapping.mapping_{index_name}")
    for row in cursor.fetchall():
        field_name = row[0]
        sqlite_col = map_field_to_column(field_name, table_name)
        if sqlite_col is not None:
            cursor.execute(f"""
                UPDATE mapping.mapping_{index_name}
                SET mapped_to = 'migrated'
                WHERE field_name = ?
            """, (field_name,))

def mark_fields_skipped(index_name, skipped_fields):
    print(f"Skipping {index_name} fields:")
    for field_name in skipped_fields:
        cursor.execute(f"""
            UPDATE mapping.mapping_{index_name}
            SET mapped_to = 'skipped'
            WHERE field_name = ?
        """, (field_name,))


for index in indices:
    inserted_fields = set()
    if "manuscript" in index:
        hits = scroll_all(index)
        for hit in hits:
            source = hit['_source']
            manuscript_id = str(source.get('id', hit['_id']))
            cursor.execute("INSERT OR IGNORE INTO manuscripts (id) VALUES (?)", (manuscript_id,))
        inserted_fields.update(get_table_columns("manuscripts"))
        mark_fields_migrated(index, inserted_fields)
        skipped = {
            "manuscript.name",
            "manuscript.name.keyword",
            "group_id"
        }
        mark_fields_skipped('dbbe_dev_verses', skipped)

    elif "occurrence" in index:
        hits = scroll_all(index)
        for hit in hits:
            source = hit['_source']
            occurrence_id = str(source.get('id', hit['_id']))
            cursor.execute("INSERT OR IGNORE INTO occurrences (id) VALUES (?)", (occurrence_id,))
        inserted_fields.update(get_table_columns("occurrences"))
        mark_fields_migrated(index, inserted_fields)
        skipped = {
            "occurrence.name",
            "occurrence.name.keyword",
            "occurrence.location",
            "occurrence.location.keyword"
        }
        mark_fields_skipped('dbbe_dev_verses', skipped)

verse_index = [idx for idx in indices if "verse" in idx]
if verse_index:
    verse_index = verse_index[0]
    hits = scroll_all(verse_index)
    table_columns = get_table_columns("verses")
    inserted_fields = set()
    for hit in hits:
        source = hit['_source']
        verse_id = str(source.get('id', hit['_id']))
        text = source.get('verse', '')
        order_in_occurrence = source.get('order', 0)
        occurrence_id = str(source.get('occurrence', {}).get('id', ''))
        manuscript_id = str(source.get('manuscript', {}).get('id', ''))
        group_id = source.get('group_id')
        group_id = str(group_id)
        if group_id is not None:
            cursor.execute(
                "INSERT OR IGNORE INTO verse_groups (id) VALUES (?)",
                (group_id,)
            )
        cursor.execute("""
            INSERT OR IGNORE INTO verses (id, text, occurrence_id, manuscript_id, order_in_occurrence, verse_group_id)
            VALUES (?, ?, ?, ?, ?,?)
        """, (verse_id, text, occurrence_id, manuscript_id, order_in_occurrence, group_id))
    for col in table_columns:
        if col in ["id", "manuscript_id", "occurrence_id", "text", "order_in_occurrence"]:
            inserted_fields.add(col)
    mark_fields_migrated(verse_index, "verses")

conn.commit()
conn.close()

print(f"Data inserted from Elasticsearch and mapping tables updated in '{main_db_path}'.")


Skipping dbbe_dev_verses fields:
Skipping dbbe_dev_verses fields:
Data inserted from Elasticsearch and mapping tables updated in 'export_data.sqlite'.


## Persons

In [16]:
import sqlite3
from elasticsearch import Elasticsearch
import uuid
es = Elasticsearch("http://localhost:19200")

main_db_path = "export_data.sqlite"
mapping_db_path = "export_data_mapping.sqlite"

conn = sqlite3.connect(main_db_path)
cursor = conn.cursor()
cursor.execute("PRAGMA foreign_keys = ON;")
cursor.execute(f"ATTACH DATABASE '{mapping_db_path}' AS mapping;")

# Helper to add column if missing
def add_column_if_missing(cursor, table_name, column_name, column_type):
    cursor.execute(f"PRAGMA table_info({table_name})")
    columns = [col[1] for col in cursor.fetchall()]
    if column_name not in columns:
        cursor.execute(f"ALTER TABLE {table_name} ADD COLUMN {column_name} {column_type}")

# Add standard person columns
person_columns = {
    "full_name": "TEXT",
    "born_date_floor_year": "TEXT",
    "born_date_ceiling_year": "TEXT",
    "death_date_floor_year": "TEXT",
    "death_date_ceiling_year": "TEXT",
    "is_dbbe_person": "BOOLEAN",
    "is_modern_person": "BOOLEAN",
    "is_historical_person": "BOOLEAN",
    "modified": "TEXT",
    "created": "TEXT",
    "public_comment":"TEXT",
    "private_comment":"TEXT"

}
for col, col_type in person_columns.items():
    add_column_if_missing(cursor, "persons", col, col_type)

# Create supporting tables
cursor.execute("""
CREATE TABLE IF NOT EXISTS roles (
    id TEXT PRIMARY KEY,
    name TEXT
)
""")
cursor.execute("""
CREATE TABLE IF NOT EXISTS management (
    id TEXT PRIMARY KEY,
    name TEXT NOT NULL
)
""")
cursor.execute("""
CREATE TABLE IF NOT EXISTS person_management (
    person_id TEXT NOT NULL,
    management_id TEXT NOT NULL,
    PRIMARY KEY (person_id, management_id),
    FOREIGN KEY (person_id) REFERENCES persons(id),
    FOREIGN KEY (management_id) REFERENCES management(id)
)
""")
cursor.execute("""
CREATE TABLE IF NOT EXISTS acknowledgements (
    id TEXT PRIMARY KEY,
    name TEXT NOT NULL
)
""")
cursor.execute("""
CREATE TABLE IF NOT EXISTS person_acknowledgement (
    person_id TEXT NOT NULL,
    acknowledgement_id TEXT NOT NULL,
    PRIMARY KEY (person_id, acknowledgement_id),
    FOREIGN KEY (person_id) REFERENCES persons(id),
    FOREIGN KEY (acknowledgement_id) REFERENCES acknowledgements(id)
)
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS self_designations (
    id TEXT PRIMARY KEY,
    name TEXT NOT NULL
)
""")
cursor.execute("""
CREATE TABLE IF NOT EXISTS person_self_designations (
    person_id TEXT NOT NULL,
    self_designation_id TEXT NOT NULL,
    PRIMARY KEY (person_id, self_designation_id),
    FOREIGN KEY (person_id) REFERENCES persons(id),
    FOREIGN KEY (self_designation_id) REFERENCES self_designations(id)
)
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS offices (
    id TEXT PRIMARY KEY,
    name TEXT NOT NULL
)
""")
cursor.execute("""
CREATE TABLE IF NOT EXISTS person_offices (
    person_id TEXT NOT NULL,
    office_id TEXT NOT NULL,
    PRIMARY KEY (person_id, office_id),
    FOREIGN KEY (person_id) REFERENCES persons(id),
    FOREIGN KEY (office_id) REFERENCES offices(id)
)
""")

# --- New identifications tables ---
cursor.execute("""
CREATE TABLE IF NOT EXISTS identifications (
    id TEXT PRIMARY KEY,
    type TEXT NOT NULL,
    identifier_value TEXT NOT NULL
)
""")
cursor.execute("""
CREATE TABLE IF NOT EXISTS person_identification (
    person_id TEXT NOT NULL,
    identification_id TEXT NOT NULL,
    PRIMARY KEY (person_id, identification_id),
    FOREIGN KEY (person_id) REFERENCES persons(id),
    FOREIGN KEY (identification_id) REFERENCES identifications(id)
)
""")

# Get person index
indices = es.cat.indices(format="json")
indices = [idx['index'] for idx in indices if idx['index'].startswith("dbbe_dev")]
person_index = next((idx for idx in indices if "person" in idx), None)

def scroll_all(index, query={"query": {"match_all": {}}, "size": 1000}):
    resp = es.search(index=index, body=query, scroll='2m')
    scroll_id = resp['_scroll_id']
    hits = resp['hits']['hits']
    all_hits = hits[:]
    while hits:
        resp = es.scroll(scroll_id=scroll_id, scroll='2m')
        hits = resp['hits']['hits']
        all_hits.extend(hits)
    return all_hits

if person_index:
    hits = scroll_all(person_index)
    print(f"Fetched {len(hits)} persons from ES")

    for hit in hits:
        source = hit['_source']
        person_id = str(source.get('id', hit['_id']))

        # Insert/update person
        data = (
            person_id,
            source.get('name', ''),
            source.get('born_date_floor_year', ''),
            source.get('born_date_ceiling_year', ''),
            source.get('death_date_floor_year', ''),
            source.get('death_date_ceiling_year', ''),
            bool(source.get('dbbe', False)),
            bool(source.get('modern', False)),
            bool(source.get('historical', False)),
            source.get('modified', ''),
            source.get('created', ''),
            source.get('public_comment', ''),
            source.get('private_comment', '')

        )

        cursor.execute("""
            INSERT OR IGNORE INTO persons
            (id, full_name, born_date_floor_year, born_date_ceiling_year,
             death_date_floor_year, death_date_ceiling_year,
             is_dbbe_person, is_modern_person, is_historical_person,
             modified, created,public_comment, private_comment)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?,?,?)
        """, data)

        # Roles
        for role in source.get('role', []):
            role_id = str(role.get('id', ''))
            role_name = role.get('name', '')
            if role_id:
                cursor.execute("INSERT OR IGNORE INTO roles (id, name) VALUES (?, ?)", (role_id, role_name))


        # Management
        for mgmt in source.get('management', []):
            mgmt_id = str(mgmt.get('id', ''))
            mgmt_name = mgmt.get('name', '')
            if mgmt_id and mgmt_name:
                cursor.execute("INSERT OR IGNORE INTO management (id, name) VALUES (?, ?)", (mgmt_id, mgmt_name))
                cursor.execute("INSERT OR IGNORE INTO person_management (person_id, management_id) VALUES (?, ?)",
                               (person_id, mgmt_id))

        # Acknowledgements
        for ack in source.get('acknowledgement', []):
            ack_id = str(ack.get('id', ''))
            ack_name = ack.get('name', '')
            if ack_id and ack_name:
                cursor.execute("INSERT OR IGNORE INTO acknowledgements (id, name) VALUES (?, ?)", (ack_id, ack_name))
                cursor.execute("INSERT OR IGNORE INTO person_acknowledgement (person_id, acknowledgement_id) VALUES (?, ?)",
                               (person_id, ack_id))

        for ack in source.get('office', []):
            office_id = str(ack.get('id', ''))
            office_name = ack.get('name', '')
            if office_id and office_name:
                cursor.execute("INSERT OR IGNORE INTO offices (id, name) VALUES (?, ?)", (office_id, office_name))
                cursor.execute("INSERT OR IGNORE INTO person_offices (person_id, office_id) VALUES (?, ?)",
                               (person_id, office_id))


        for self_designation in source.get('self_designation', []):
            self_designation_id = str(self_designation.get('id', ''))
            self_designation_name = self_designation.get('name', '')
            if self_designation_id and self_designation_name:
                cursor.execute("INSERT OR IGNORE INTO self_designations (id, name) VALUES (?, ?)", (self_designation_id, self_designation_name))
                cursor.execute("INSERT OR IGNORE INTO person_self_designations (person_id, self_designation_id) VALUES (?, ?)",
                               (person_id, self_designation_id))

        # --- Identifications ---
        IDENT_TYPE_MAP = {
        "plp": "plp",
            "pmbz": "pmbz",
            "rgk": "rgk",
            "vgh": "vgh",
            "pbw": "pbw",
            "pbe": "pbe",
            "pinakes_person": "pinakes",
        }
        for es_field, sql_type in IDENT_TYPE_MAP.items():
            for identifier in source.get(es_field, []):
                if not identifier:
                    continue

                ident_id = str(uuid.uuid4())

                cursor.execute(
                    """
                    INSERT OR IGNORE INTO identifications
                    (id, type, identifier_value)
                    VALUES (?, ?, ?)
                    """,
                    (ident_id, sql_type, identifier)
                )

                cursor.execute(
                    """
                    INSERT OR IGNORE INTO person_identification
                    (person_id, identification_id)
                    VALUES (?, ?)
                    """,
                    (person_id, ident_id)
                )

    mapping_table = f"mapping.mapping_{person_index}"
    cursor.execute(f"SELECT field_name FROM {mapping_table}")
    migrated_fields = [
        "id", "name",
        "born_date_floor_year","born_date_ceiling_year",
        "death_date_floor_year","death_date_ceiling_year",
        "dbbe","modern","historical",
        "modified","created",
        "role.id","role.name","role",
        "management.id","management.name",
        "acknowledgement.id","acknowledgement.name",
        "plp","pmbz","rgk","vgh","pbw","pbe","self_designation.id","self_designation","self_designation.name",
        "pinakes_person","acknowledgement","public_comment","private_comment","management"
    ]
    skip_fields = [
        "role.id_name","role.id_name.keyword",
        "role.name.keyword",
        "management.id_name","management.id_name.keyword",
        "acknowledgement.id_name","acknowledgement.id_name.keyword",
        "plp_available","plp.keyword","pmbz_available","pmbz.keyword",
        "rgk_available","rgk.keyword","vgh_available","vgh.keyword","pwb_available","pwb.keyword","self_designation.name.keyword","self_designation.id_name","self_designation.id_name.keyword","pbe_available","pbe.keyword",
        "pinakes_person_available","pinakes_person.keyword","pbw_available","pbw.keyword","acknowledgement.name.keyword","management.name.keyword","public_comment.keyword","private_comment.keyword","name.keyword",
        "public_comment.keyword","private_comment.keyword"
    ]
    for (f,) in cursor.fetchall():
        if f in migrated_fields:
            cursor.execute(f"UPDATE {mapping_table} SET mapped_to='migrated' WHERE field_name=?", (f,))
        elif f in skip_fields:
            cursor.execute(f"UPDATE {mapping_table} SET mapped_to='skipped' WHERE field_name=?", (f,))

conn.commit()
conn.close()
print("Persons, roles, managements, acknowledgements, and identifications updated; mapping table updated.")


Fetched 3746 persons from ES
Persons, roles, managements, acknowledgements, and identifications updated; mapping table updated.


## Types


In [17]:
mapping = {
            "id": "migrated",
            "text_stemmer": "migrated",
            "text_original": "migrated",
            "lemma": "migrated",
            "public_comment": "migrated",
            "private_comment": "migrated",
            "incipit": "migrated",
            "created": "migrated",
            "modified": "migrated",
            "occurrence_ids": "migrated",
            "tag.id": "migrated",
            "tag.name": "migrated",
            "management.id": "migrated",
            "management.name": "migrated",
            "acknowledgement.id": "migrated",
            "acknowledgement.name": "migrated",
            "tag.id_name": "skipped",
            "management.id_name": "skipped",
            "management.id_name.keyword": "skipped",
            "acknowledgement.id_name": "skipped",
            "acknowledgement.id_name.keyword": "skipped",
            "subject.id":"migrated",
            "subject.name":"migrated",
            "subject.name.keyword":"skipped",
            "subject.id_name":"skipped",
            "subject.id_name.keyword":"skipped",
            "creator":"migrated",
            "creator.id":"skipped",
            "creator.id_name":"skipped",
            "creator.id_name.keyword":"skipped",
            "creator.name":"skipped",
            "creator.name.keyword":"skipped",
            "creator_public":"skipped",
            "creator_public.id":"skipped",
            "creator_public.id_name":"skipped",
            "creator_public.id_name.keyword":"skipped",
            "creator_public.name":"skipped",
            "creator_public.name.keyword":"skipped",
            "poet":"migrated",
            "poet.id":"skipped",
            "poet.id_name":"skipped",
            "poet.id_name.keyword":"skipped",
            "poet.name":"skipped",
            "poet.name.keyword":"skipped",
            "poet_public":"skipped",
            "poet_public.id":"skipped",
            "poet_public.id_name":"skipped",
            "poet_public.id_name.keyword":"skipped",
            "poet_public.name":"skipped",
            "poet_public.name.keyword":"skipped",
            "contributor":"migrated",
            "contributor.id":"skipped",
            "contributor.id_name":"skipped",
            "contributor.id_name.keyword":"skipped",
            "contributor.name":"skipped",
            "contributor.name.keyword":"skipped",
            "contributor_public":"skipped",
            "contributor_public.id":"skipped",
            "contributor_public.id_name":"skipped",
            "contributor_public.id_name.keyword":"skipped",
            "contributor_public.name":"skipped",
            "contributor_public.name.keyword":"skipped",
            "critical_status.id":"migrated",
            "critical_status.name":"migrated",
            "critical_status.name.keyword":"skipped",
            "critical_status.id_name":"skipped",
            "criitical_status.id_name.keyword":"skipped"
        }

In [18]:
import sqlite3
from elasticsearch import Elasticsearch
import psycopg2

pg_conn = psycopg2.connect(
    host="localhost",
    port=15432,
    dbname="db_dbbe_dev",
    user="db_dbbe_dev",
    password="db_dbbe_dev"
)
pg_cursor = pg_conn.cursor()


es = Elasticsearch("http://localhost:19200")
db_path = "export_data.sqlite"
mapping_db_path = "export_data_mapping.sqlite"

conn = sqlite3.connect(db_path, timeout=60, isolation_level=None)
cursor = conn.cursor()
cursor.execute("PRAGMA journal_mode = WAL;")
cursor.execute("PRAGMA busy_timeout = 60000;")
cursor.execute("PRAGMA foreign_keys = ON;")
cursor.execute(f"ATTACH DATABASE '{mapping_db_path}' AS mapping;")

def add_column_if_missing(cursor, table_name, column_name, column_type):
    cursor.execute(f"PRAGMA table_info({table_name})")
    columns = [col[1] for col in cursor.fetchall()]
    if column_name not in columns:
        cursor.execute(f"ALTER TABLE {table_name} ADD COLUMN {column_name} {column_type}")

add_column_if_missing(cursor, "types", "text_stemmer", "TEXT")
add_column_if_missing(cursor, "types", "text_original", "TEXT")
add_column_if_missing(cursor, "types", "lemma", "TEXT")
add_column_if_missing(cursor, "types", "incipit", "TEXT")
add_column_if_missing(cursor, "types", "created", "TEXT")
add_column_if_missing(cursor, "types", "modified", "TEXT")
add_column_if_missing(cursor, "types", "public_comment", "TEXT")
add_column_if_missing(cursor, "types", "private_comment", "TEXT")
add_column_if_missing(cursor, "types", "title", "TEXT")

cursor.execute("""
CREATE TABLE IF NOT EXISTS metres (
    id TEXT PRIMARY KEY,
    name TEXT
)
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS genres (
    id TEXT PRIMARY KEY,
    name TEXT
)
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS text_statuses (
    id TEXT PRIMARY KEY,
    name TEXT
)
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS subjects (
    id TEXT PRIMARY KEY,
    name TEXT
)
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS acknowledgements (
    id TEXT PRIMARY KEY,
    name TEXT NOT NULL
)
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS tags (
    id TEXT PRIMARY KEY,
    name TEXT
)
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS editorial_statuses (
    id TEXT PRIMARY KEY,
    name TEXT
)
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS type_management (
    type_id TEXT NOT NULL,
    management_id TEXT NOT NULL,
    PRIMARY KEY (type_id, management_id),
    FOREIGN KEY (type_id) REFERENCES types(id),
    FOREIGN KEY (management_id) REFERENCES management(id)
)
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS type_metre (
    type_id TEXT NOT NULL,
    metre_id TEXT NOT NULL,
    PRIMARY KEY (type_id, metre_id),
    FOREIGN KEY (type_id) REFERENCES types(id),
    FOREIGN KEY (metre_id) REFERENCES metres(id)
)
""")


cursor.execute("""
CREATE TABLE IF NOT EXISTS type_genre (
    type_id TEXT NOT NULL,
    genre_id TEXT NOT NULL,
    PRIMARY KEY (type_id, genre_id),
    FOREIGN KEY (type_id) REFERENCES types(id),
    FOREIGN KEY (genre_id) REFERENCES genres(id)
)
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS type_related_types (
    type_id TEXT NOT NULL,
    related_type_id TEXT NOT NULL,
    relation_definition_id TEXT NOT NULL,

    PRIMARY KEY (type_id, related_type_id, relation_definition_id),

    FOREIGN KEY (type_id) REFERENCES types(id),
    FOREIGN KEY (related_type_id) REFERENCES types(id),
    FOREIGN KEY (relation_definition_id)
        REFERENCES type_relation_definitions(id),

    CHECK (type_id <> related_type_id)
);
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS type_text_statuses (
    type_id TEXT NOT NULL,
    text_status_id TEXT NOT NULL,
    PRIMARY KEY (type_id, text_status_id),
    FOREIGN KEY (type_id) REFERENCES types(id),
    FOREIGN KEY (text_status_id) REFERENCES text_statuses(id)
)
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS type_tags (
    type_id TEXT NOT NULL,
    tag_id TEXT NOT NULL,
    PRIMARY KEY (type_id, tag_id),
    FOREIGN KEY (type_id) REFERENCES types(id),
    FOREIGN KEY (tag_id) REFERENCES tags(id)
)
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS type_occurrences (
    type_id TEXT NOT NULL,
    occurrence_id TEXT NOT NULL,
    PRIMARY KEY (type_id, occurrence_id),
    FOREIGN KEY (type_id) REFERENCES types(id),
    FOREIGN KEY (occurrence_id) REFERENCES occurrences(id)
)
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS type_relation_definitions (
    id TEXT PRIMARY KEY,
    definition TEXT NOT NULL UNIQUE
)
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS type_acknowledgement (
    type_id TEXT NOT NULL,
    acknowledgement_id TEXT NOT NULL,
    PRIMARY KEY (type_id, acknowledgement_id),
    FOREIGN KEY (type_id) REFERENCES types(id),
    FOREIGN KEY (acknowledgement_id) REFERENCES acknowledgements(id)
)
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS type_subject (
    type_id TEXT NOT NULL,
    subject_id TEXT NOT NULL,
    PRIMARY KEY (type_id, subject_id),
    FOREIGN KEY (type_id) REFERENCES types(id),
    FOREIGN KEY (subject_id) REFERENCES subjects(id)
)
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS type_editorial_status (
    type_id TEXT NOT NULL,
    editorial_status_id TEXT NOT NULL,
    PRIMARY KEY (type_id, editorial_status_id),
    FOREIGN KEY (type_id) REFERENCES types(id),
    FOREIGN KEY (editorial_status_id) REFERENCES editorial_statuses(id)
)
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS type_person_roles (
    type_id TEXT NOT NULL,
    person_id TEXT NOT NULL,
    role_id TEXT NOT NULL,
    PRIMARY KEY (type_id, person_id, role_id),
    FOREIGN KEY (type_id) REFERENCES types(id),
    FOREIGN KEY (person_id) REFERENCES persons(id),
    FOREIGN KEY (role_id) REFERENCES roles(id)
)
""")

indices = es.cat.indices(format="json")
indices = [idx['index'] for idx in indices if idx['index'].startswith("dbbe_dev")]
type_index = next((idx for idx in indices if "type" in idx), None)

def scroll_all(index, query={"query": {"match_all": {}}, "size": 1000}):
    resp = es.search(index=index, body=query, scroll='2m')
    scroll_id = resp['_scroll_id']
    hits = resp['hits']['hits']
    all_hits = hits[:]
    while hits:
        resp = es.scroll(scroll_id=scroll_id, scroll='2m')
        hits = resp['hits']['hits']
        all_hits.extend(hits)
    return all_hits

def get_role_id(role_name):
    cursor.execute("SELECT id FROM roles WHERE LOWER(name)=LOWER(?)", (role_name,))
    row = cursor.fetchone()
    return row[0] if row else None

if type_index:
    hits = scroll_all(type_index)

    cursor.execute("BEGIN TRANSACTION")
    batch_count = 0

    for hit in hits:
        source = hit['_source']
        type_id = str(source.get('id', hit['_id']))
        text_stemmer = source.get('text_stemmer', '')
        text_original = source.get('text_original', '')
        lemma = source.get('lemma', '')
        incipit = source.get('incipit', '')
        created = source.get('created', '')
        modified = source.get('modified', '')
        public_comment = source.get('public_comment', '')
        private_comment = source.get('private_comment', '')
        title_GR_original = source.get('title_GR_original', '')

        cursor.execute("""
            INSERT OR IGNORE INTO types
            (id, text_stemmer, text_original, lemma, incipit, created, modified, public_comment, private_comment, title)
            VALUES (?, ?, ?, ?, ?, ?, ?, ? , ? ,?)
        """, (type_id, text_stemmer, text_original, lemma, incipit, created, modified, public_comment, private_comment, title_GR_original))

        for occ_id in source.get('occurrence_ids', []):
            cursor.execute("""
                INSERT OR IGNORE INTO type_occurrences (type_id, occurrence_id)
                VALUES (?, ?)
            """, (type_id, str(occ_id)))

        for tag in source.get('tag', []):
            tag_id = str(tag.get('id', ''))
            tag_name = tag.get('name', '')
            if tag_id:
                cursor.execute("INSERT OR IGNORE INTO tags (id, name) VALUES (?, ?)", (tag_id, tag_name))
                cursor.execute("INSERT OR IGNORE INTO type_tags (type_id, tag_id) VALUES (?, ?)", (type_id, tag_id))

        for mgmt in source.get('management', []):
            mgmt_id = str(mgmt.get('id', ''))
            mgmt_name = mgmt.get('name', '')
            if mgmt_id and mgmt_name:
                cursor.execute("INSERT OR IGNORE INTO management (id, name) VALUES (?, ?)", (mgmt_id, mgmt_name))
                cursor.execute("INSERT OR IGNORE INTO type_management (type_id, management_id) VALUES (?, ?)",
                               (type_id, mgmt_id))

        for ack in source.get('acknowledgement', []):
            ack_id = str(ack.get('id', ''))
            ack_name = ack.get('name', '')
            if ack_id and ack_name:
                cursor.execute("INSERT OR IGNORE INTO acknowledgements (id, name) VALUES (?, ?)", (ack_id, ack_name))
                cursor.execute("INSERT OR IGNORE INTO type_acknowledgement (type_id, acknowledgement_id) VALUES (?, ?)",
                               (type_id, ack_id))

        ts = source.get("text_status")
        if isinstance(ts, dict):
            ts_id = str(ts.get("id", ""))
            ts_name = ts.get("name", "")
            if ts_id:
                cursor.execute(
                    "INSERT OR IGNORE INTO text_statuses (id, name) VALUES (?, ?)",
                    (ts_id, ts_name)
                )
                cursor.execute(
                    "INSERT OR IGNORE INTO type_text_statuses (type_id, text_status_id) VALUES (?, ?)",
                    (type_id, ts_id)
                )

        ts = source.get("critical_status")
        if isinstance(ts, dict):
            es_id = str(ts.get("id", ""))
            es_name = ts.get("name", "")
            if es_id:
                cursor.execute(
                    "INSERT OR IGNORE INTO editorial_statuses (id, name) VALUES (?, ?)",
                    (es_id, es_name)
                )
                cursor.execute(
                    "INSERT OR IGNORE INTO type_editorial_status (type_id, editorial_status_id) VALUES (?, ?)",
                    (type_id, es_id)
                )

        genres = source.get("genre", [])
        if isinstance(genres, dict):
            genres = [genres]
        for g in genres:
            genre_id = str(g.get("id", ""))
            genre_name = g.get("name", "")
            if genre_id:
                cursor.execute("INSERT OR IGNORE INTO genres (id, name) VALUES (?, ?)", (genre_id, genre_name))
                cursor.execute("INSERT OR IGNORE INTO type_genre (type_id, genre_id) VALUES (?, ?)", (type_id, genre_id))

        metres = source.get("metre", [])
        if isinstance(metres, dict):
            metres = [metres]
        for m in metres:
            metre_id = str(m.get("id", ""))
            metre_name = m.get("name", "")
            if metre_id:
                cursor.execute("INSERT OR IGNORE INTO metres (id, name) VALUES (?, ?)", (metre_id, metre_name))
                cursor.execute("INSERT OR IGNORE INTO type_metre (type_id, metre_id) VALUES (?, ?)", (type_id, metre_id))


        subjects = source.get('subject', [])
        seen_subject_ids = set()
        for subj in subjects:
            subj_id = str(subj.get('id', ''))
            subj_name = subj.get('name', '')
            if not subj_id or subj_id in seen_subject_ids:
                continue
            seen_subject_ids.add(subj_id)

            cursor.execute(
                "INSERT OR IGNORE INTO subjects (id, name) VALUES (?, ?)",
                (subj_id, subj_name)
            )
            cursor.execute(
                "INSERT OR IGNORE INTO type_subject (type_id, subject_id) VALUES (?, ?)",
                (type_id, subj_id)
            )

        role_field_to_role_name = {
            "contributor": "Contributor",
            "creator": "Creator",
            "person_subject": "Subject",
            "poet": "Poet",
     }
        for role_field, role_name_in_table in role_field_to_role_name.items():
            role_id = get_role_id(role_name_in_table)
            if not role_id:
                continue
            persons = source.get(role_field, [])
            if isinstance(persons, dict):
                persons = [persons]
            elif not isinstance(persons, list):
                persons = []
            for p in persons:
                person_id = str(p.get("id",""))
                if not person_id:
                    continue

                cursor.execute("SELECT 1 FROM persons WHERE id=?", (person_id,))
                if cursor.fetchone() is None:
                    continue

                if role_id is None:
                    continue

                cursor.execute(
                    "INSERT OR IGNORE INTO type_person_roles (type_id, person_id, role_id) VALUES (?,?,?)",
                    (type_id, person_id, role_id)
                )
        batch_count += 1
        if batch_count % 1000 == 0:
            cursor.execute("COMMIT")
            cursor.execute("BEGIN TRANSACTION")
            print(f"Processed {batch_count} types...")

    cursor.execute("COMMIT")
    print(f"Total types processed: {batch_count}")

    pg_cursor.execute("""
        SELECT
            f.subject_identity,
            f.object_identity,
            ft.idfactoid_type,
            ft.type
        FROM data.factoid f
        JOIN data.factoid_type ft
            ON f.idfactoid_type = ft.idfactoid_type
        WHERE ft.group = 'reconstructed_poem_related_to_reconstructed_poem'
    """)

    relations = pg_cursor.fetchall()

    cursor.execute("BEGIN TRANSACTION")

    for _, _, rel_def_id, rel_code in relations:
        cursor.execute("""
            INSERT OR IGNORE INTO type_relation_definitions
            (id, definition)
            VALUES (?, ?)
        """, (str(rel_def_id), rel_code))

    cursor.execute("COMMIT")

    cursor.execute("BEGIN TRANSACTION")

    for type_id, related_type_id, rel_def_id, _ in relations:
        cursor.execute("""
            INSERT OR IGNORE INTO type_related_types
            (type_id, related_type_id, relation_definition_id)
            VALUES (?, ?, ?)
        """, (
            str(type_id),
            str(related_type_id),
            str(rel_def_id)
        ))

    cursor.execute("COMMIT")

    pg_cursor.close()
    pg_conn.close()

    def map_type_field_to_column(field_name):
        if field_name.startswith("genre") or field_name.startswith("metre"):
            return "skipped"
        return mapping.get(field_name)

    cursor.execute("BEGIN TRANSACTION")
    mapping_table = f"mapping.mapping_{type_index}"
    cursor.execute(f"SELECT field_name FROM {mapping_table}")
    for row in cursor.fetchall():
        field_name = row[0]
        status = map_type_field_to_column(field_name)
        if status:
            cursor.execute(
                f"UPDATE {mapping_table} SET mapped_to = ? WHERE field_name = ?",
                (status, field_name)
            )
    cursor.execute("COMMIT")

conn.close()
print(f"Types, type_occurrences, tags, managements, and acknowledgements updated; mapping table updated.")


Processed 1000 types...
Processed 2000 types...
Processed 3000 types...
Processed 4000 types...
Processed 5000 types...
Total types processed: 5284
Types, type_occurrences, tags, managements, and acknowledgements updated; mapping table updated.


## Occurrences

In [19]:
migrated = [
            "manuscript.id","contributor.id","transcriber.id","scribe.id","person_subject.id", "genre","metre",
            "genre.id","genre.name","metre.id","metre.name",
            "id","created","modified","public_comment","private_comment",
            "is_dbbe","incipit","text_stemmer","text_original","location",
            "date_floor_year","date_ceiling_year","palaeographical_info","contextual_info",
            "management.id","management.name","scribe",
            "acknowledgement.id","acknowledgement.name" ,"acknowledgement","text_status",
            "text_status.id",
            "text_status.name", "subject.id","subject.name","person_subject","patron","management","subject"

        ]
skipped = [
            "management.id_name","management.id_name.keyword",
            "acknowledgement.id_name","acknowledgement.id_name.keyword","text_status.id_name",
                "text_status.id_name.keyword",
                "text_status.name.keyword","subject.id_name","subject.name.keyword","subject.id_name.keyword",
            "patron.id",
            "patron.id_name.keyword",
            "patron.id_name",
            "patron.name.keyword",
            "patron.name",
"patron_public.id",
"patron_public.id_name.keyword",
"patron_public.id_name",
"patron_public.name.keyword",
"patron_public.name",
"patron_public",
"person_subject_public.id",
"person_subject_public.id_name.keyword",
"person_subject_public.id_name",
"person_subject_public.name.keyword",
"person_subject_public.name",
"person_subject_public",
"creator.id",
"creator.id_name.keyword",
"creator.id_name",
"creator.name.keyword",
"creator.name",
"creator_public.id",
"creator_public.id_name.keyword",
"creator_public.id_name",
"creator_public.name.keyword",
"creator_public.name",
            "incipit.keyword",
            "management.name.keyword"
        ]

In [20]:

def get_related_occurrences(occ_id, pg_cursor):
    pg_cursor.execute("""
        SELECT count(a.id) as count, b.idoriginal_poem
        FROM data.original_poem_verse a
        INNER JOIN data.original_poem_verse b ON a.idgroup = b.idgroup
        WHERE a.idoriginal_poem = %s AND b.idoriginal_poem <> a.idoriginal_poem
        GROUP BY b.idoriginal_poem

        UNION

        SELECT 0 as count, fb.subject_identity
        FROM data.factoid fa
        INNER JOIN data.factoid_type fta ON fa.idfactoid_type = fta.idfactoid_type
        INNER JOIN data.factoid fb ON fa.object_identity = fb.object_identity
        INNER JOIN data.factoid_type ftb ON fb.idfactoid_type = ftb.idfactoid_type
        WHERE fa.subject_identity = %s
        AND fta.type = 'reconstruction of'
        AND ftb.type = 'reconstruction of'
        AND fb.subject_identity <> fa.subject_identity
        AND fb.subject_identity NOT IN (
            SELECT b.idoriginal_poem
            FROM data.original_poem_verse a
            INNER JOIN data.original_poem_verse b ON a.idgroup = b.idgroup
            WHERE a.idoriginal_poem = %s
            AND b.idoriginal_poem <> a.idoriginal_poem
            GROUP BY b.idoriginal_poem
        )
    """, (occ_id, occ_id, occ_id))
    return pg_cursor.fetchall()

In [21]:
import sqlite3
from elasticsearch import Elasticsearch

pg_conn = psycopg2.connect(
    host="localhost",
    port=15432,
    dbname="db_dbbe_dev",
    user="db_dbbe_dev",
    password="db_dbbe_dev"
)
pg_cursor = pg_conn.cursor()

es = Elasticsearch("http://localhost:19200")

main_db_path = "export_data.sqlite"
mapping_db_path = "export_data_mapping.sqlite"

conn = sqlite3.connect(main_db_path, timeout=60, isolation_level=None)
cursor = conn.cursor()
cursor.execute("PRAGMA journal_mode = WAL;")
cursor.execute("PRAGMA busy_timeout = 60000;")
cursor.execute("PRAGMA foreign_keys = ON;")
cursor.execute(f"ATTACH DATABASE '{mapping_db_path}' AS mapping;")

# Add missing columns
def add_column_if_missing(cursor, table_name, column_name, column_type):
    cursor.execute(f"PRAGMA table_info({table_name})")
    cols = [c[1] for c in cursor.fetchall()]
    if column_name not in cols:
        cursor.execute(f"ALTER TABLE {table_name} ADD COLUMN {column_name} {column_type}")

columns_to_add = [
    ("created","TEXT"),("modified","TEXT"),
    ("public_comment","TEXT"),("private_comment","TEXT"),
    ("is_dbbe","BOOLEAN"),
    ("incipit","TEXT"),("text_stemmer","TEXT"),("text_original","TEXT"),
    ("location","TEXT"),
    ("date_floor_year","TEXT"),("date_ceiling_year","TEXT"),
    ("palaeographical_info","TEXT"),("contextual_info","TEXT"),
    ("manuscript_id","TEXT"),
    ("title","TEXT")

]

for c,t in columns_to_add:
    add_column_if_missing(cursor,"occurrences",c,t)

# Ensure tables exist
cursor.execute("""
CREATE TABLE IF NOT EXISTS occurrence_person_roles (
    occurrence_id TEXT NOT NULL,
    person_id TEXT NOT NULL,
    role_id TEXT NOT NULL,
    PRIMARY KEY (occurrence_id, person_id, role_id),
    FOREIGN KEY (occurrence_id) REFERENCES occurrences(id),
    FOREIGN KEY (person_id) REFERENCES persons(id),
    FOREIGN KEY (role_id) REFERENCES roles(id)
)
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS occurrence_relation_definitions (
    id TEXT PRIMARY KEY,
    definition TEXT NOT NULL UNIQUE
)
""")


cursor.execute("""
INSERT OR IGNORE INTO occurrence_relation_definitions (id, definition) VALUES
('0', 'verse_related'),
('1', 'type_related');

""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS occurrence_related_occurrences (
    occurrence_id TEXT NOT NULL,
    related_occurrence_id TEXT NOT NULL,
    relation_definition_id TEXT NOT NULL,
    PRIMARY KEY (occurrence_id, related_occurrence_id, relation_definition_id),
    FOREIGN KEY (occurrence_id) REFERENCES occurrences(id),
    FOREIGN KEY (related_occurrence_id) REFERENCES occurrences(id),
    FOREIGN KEY (relation_definition_id) REFERENCES occurrence_relation_definitions(id),
    CHECK (occurrence_id <> related_occurrence_id)
)
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS genres (
    id TEXT PRIMARY KEY,
    name TEXT
)
""")
cursor.execute("""
CREATE TABLE IF NOT EXISTS occurrence_genres (
    occurrence_id TEXT NOT NULL,
    genre_id TEXT NOT NULL,
    PRIMARY KEY (occurrence_id, genre_id),
    FOREIGN KEY (occurrence_id) REFERENCES occurrences(id),
    FOREIGN KEY (genre_id) REFERENCES genres(id)
)
""")
cursor.execute("""
CREATE TABLE IF NOT EXISTS metres (
    id TEXT PRIMARY KEY,
    name TEXT
)
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS occurrence_metres (
    occurrence_id TEXT NOT NULL,
    metre_id TEXT NOT NULL,
    PRIMARY KEY (occurrence_id, metre_id),
    FOREIGN KEY (occurrence_id) REFERENCES occurrences(id),
    FOREIGN KEY (metre_id) REFERENCES metres(id)
)
""")
cursor.execute("""
CREATE TABLE IF NOT EXISTS occurrence_management (
    occurrence_id TEXT NOT NULL,
    management_id TEXT NOT NULL,
    PRIMARY KEY (occurrence_id, management_id),
    FOREIGN KEY (occurrence_id) REFERENCES occurrences(id),
    FOREIGN KEY (management_id) REFERENCES management(id)
)
""")
cursor.execute("""
CREATE TABLE IF NOT EXISTS acknowledgements (
    id TEXT PRIMARY KEY,
    name TEXT NOT NULL
)
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS occurrence_acknowledgement (
    occurrence_id TEXT NOT NULL,
    acknowledgement_id TEXT NOT NULL,
    PRIMARY KEY (occurrence_id, acknowledgement_id),
    FOREIGN KEY (occurrence_id) REFERENCES occurrences(id),
    FOREIGN KEY (acknowledgement_id) REFERENCES acknowledgements(id)
)
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS text_statuses (
    id TEXT PRIMARY KEY,
    name TEXT
)
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS subjects (
    id TEXT PRIMARY KEY,
    name TEXT
)
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS occurrence_text_statuses (
    occurrence_id TEXT NOT NULL,
    text_status_id TEXT NOT NULL,
    PRIMARY KEY (occurrence_id, text_status_id),
    FOREIGN KEY (occurrence_id) REFERENCES occurrences(id),
    FOREIGN KEY (text_status_id) REFERENCES text_statuses(id)
)
""")



cursor.execute("""
CREATE TABLE IF NOT EXISTS occurrence_subject (
    occurrence_id TEXT NOT NULL,
    subject_id TEXT NOT NULL,
    PRIMARY KEY (occurrence_id, subject_id),
    FOREIGN KEY (occurrence_id) REFERENCES occurrences(id),
    FOREIGN KEY (subject_id) REFERENCES subjects(id)
)
""")


# Get occurrences index
indices = es.cat.indices(format="json")
indices = [i["index"] for i in indices if i["index"].startswith("dbbe_dev")]
occ_index = next((i for i in indices if i.endswith("occurrences")), None)

def scroll_all(index):
    """Scroll through all documents in the ES index."""
    resp = es.search(index=index, body={"query":{"match_all":{}},"size":500}, scroll="2m")
    scroll_id = resp["_scroll_id"]
    hits = resp["hits"]["hits"]
    all_hits = hits[:]
    while True:
        if not hits:
            break
        resp = es.scroll(scroll_id=scroll_id, scroll="2m")
        scroll_id = resp["_scroll_id"]
        hits = resp["hits"]["hits"]
        if not hits:
            break
        all_hits.extend(hits)
        if len(all_hits) % 1000 == 0:
            print(f"Fetched {len(all_hits)} occurrences from ES...")
    return all_hits

def get_role_id(role_name):
    cursor.execute("SELECT id FROM roles WHERE LOWER(name)=LOWER(?)", (role_name,))
    row = cursor.fetchone()
    return row[0] if row else None

if occ_index:
    hits = scroll_all(occ_index)
    print(f"Total occurrences fetched: {len(hits)}")

    cursor.execute("BEGIN")
    batch_count = 0

    for h in hits:
        s = h["_source"]
        oid = str(s.get("id", h["_id"]))
        mid = str(s.get("manuscript",{}).get("id",""))

        # Update occurrence main fields
        cursor.execute("""
        UPDATE occurrences SET
            created=?, modified=?, public_comment=?, private_comment=?,
            is_dbbe=?, incipit=?, text_stemmer=?, text_original=?,
            location=?, date_floor_year=?, date_ceiling_year=?,
            palaeographical_info=?, contextual_info=?, manuscript_id=?, title=?
        WHERE id=?
        """,(
            s.get("created",""), s.get("modified",""),
            s.get("public_comment",""), s.get("private_comment",""),
            bool(s.get("is_dbbe",False)),
            s.get("incipit",""), s.get("text_stemmer",""), s.get("text_original",""),
            s.get("location",""), s.get("date_floor_year",""), s.get("date_ceiling_year",""),
            s.get("palaeographical_info",""), s.get("contextual_info",""),
            mid,                    # <- manuscript_id
            s.get("title_original",""),  # <- title column
            oid
        ))
        # Genres & Metres
        for g in s.get("genre", []):
            gid = str(g.get("id",""))
            if gid:
                cursor.execute("INSERT OR IGNORE INTO genres VALUES (?,?)",(gid,g.get("name","")))
                cursor.execute("INSERT OR IGNORE INTO occurrence_genres VALUES (?,?)",(oid,gid))
        for m in s.get("metre", []):
            mid2 = str(m.get("id",""))
            if mid2:
                cursor.execute("INSERT OR IGNORE INTO metres VALUES (?,?)",(mid2,m.get("name","")))
                cursor.execute("INSERT OR IGNORE INTO occurrence_metres VALUES (?,?)",(oid,mid2))
        for ack in s.get("acknowledgement", []):
            ack_id = str(ack.get("id", ""))
            ack_name = ack.get("name", "")
            if ack_id and ack_name:
                cursor.execute(
                    "INSERT OR IGNORE INTO acknowledgements (id, name) VALUES (?, ?)",
                    (ack_id, ack_name)
                )
                cursor.execute(
                    "INSERT OR IGNORE INTO occurrence_acknowledgement (occurrence_id, acknowledgement_id) VALUES (?, ?)",
                    (oid, ack_id)
                )
        subjects = s.get("subject", [])
        seen_subject_ids = set()

        for subj in subjects:
            subj_id = str(subj.get("id", ""))
            subj_name = subj.get("name", "")
            if not subj_id or subj_id in seen_subject_ids:
                continue

            seen_subject_ids.add(subj_id)

            cursor.execute(
                "INSERT OR IGNORE INTO subjects (id, name) VALUES (?, ?)",
                (subj_id, subj_name)
            )
            cursor.execute(
                "INSERT OR IGNORE INTO occurrence_subject (occurrence_id, subject_id) VALUES (?, ?)",
                (oid, subj_id)
            )

        ts = s.get("text_status")
        if isinstance(ts, dict):
            ts_id = str(ts.get("id", ""))
            ts_name = ts.get("name", "")
            if ts_id:
                cursor.execute(
                    "INSERT OR IGNORE INTO text_statuses (id, name) VALUES (?, ?)",
                    (ts_id, ts_name)
                )
                cursor.execute(
                    "INSERT OR IGNORE INTO occurrence_text_statuses (occurrence_id, text_status_id) VALUES (?, ?)",
                    (oid, ts_id)
                )


        role_field_to_role_name = {
            "contributor": "Contributor",
            "transcriber": "Transcriber",
            "scribe": "Scribe",
            "person_subject": "Subject",
            "creator": "Creator",
            "patron": "Patron"        }
        for role_field, role_name_in_table in role_field_to_role_name.items():
            role_id = get_role_id(role_name_in_table)
            if not role_id:
                continue
            persons = s.get(role_field, [])
            if isinstance(persons, dict):
                persons = [persons]
            elif not isinstance(persons, list):
                persons = []
            for p in persons:
                person_id = str(p.get("id",""))
                if person_id:
                    cursor.execute("SELECT 1 FROM persons WHERE id=?", (person_id,))
                    if cursor.fetchone():
                        cursor.execute(
                            "INSERT OR IGNORE INTO occurrence_person_roles (occurrence_id, person_id, role_id) VALUES (?,?,?)",
                            (oid, person_id, role_id)
                        )

        # Managements
        for mgmt in s.get("management", []):
            mgmt_id = str(mgmt.get("id",""))
            mgmt_name = mgmt.get("name","")
            if mgmt_id and mgmt_name:
                cursor.execute("INSERT OR IGNORE INTO management (id,name) VALUES (?,?)",(mgmt_id,mgmt_name))
                cursor.execute("INSERT OR IGNORE INTO occurrence_management (occurrence_id, management_id) VALUES (?,?)",(oid,mgmt_id))

        batch_count += 1
        if batch_count % 1000 == 0:
            cursor.execute("COMMIT")
            cursor.execute("BEGIN")
            print(f"Committed {batch_count} occurrences...")

    cursor.execute("COMMIT")
    print(f"Total occurrences processed and committed: {batch_count}")


    cursor.execute("BEGIN TRANSACTION")

    # 1️⃣ Get all occurrence IDs from SQLite
    cursor.execute("SELECT id FROM occurrences")
    occurrence_ids = [row[0] for row in cursor.fetchall()]
    occurrence_ids_int = [int(x) for x in occurrence_ids]
    if not occurrence_ids:
        print("No occurrences found, skipping related occurrences migration.")
    else:
        print(f"Found {len(occurrence_ids)} occurrences. Fetching related occurrences from Postgres...")

        pg_cursor.execute("""
            SELECT a.idoriginal_poem AS occurrence_id,
                   b.idoriginal_poem AS related_occurrence_id,
                   '0' AS relation_definition_id
            FROM data.original_poem_verse a
            JOIN data.original_poem_verse b
              ON a.idgroup = b.idgroup
            WHERE a.idoriginal_poem = ANY(%s)
              AND b.idoriginal_poem <> a.idoriginal_poem
            GROUP BY a.idoriginal_poem, b.idoriginal_poem
        """, (occurrence_ids_int,))
        verse_rows = pg_cursor.fetchall()

        pg_cursor.execute("""
            SELECT fa.subject_identity AS occurrence_id,
                   fb.subject_identity AS related_occurrence_id,
                   '1' AS relation_definition_id
            FROM data.factoid fa
            JOIN data.factoid_type fta ON fa.idfactoid_type = fta.idfactoid_type
            JOIN data.factoid fb ON fa.object_identity = fb.object_identity
            JOIN data.factoid_type ftb ON fb.idfactoid_type = ftb.idfactoid_type
            JOIN data.original_poem opb ON fb.subject_identity = opb.identity
            WHERE fa.subject_identity = ANY(%s)
              AND fta.type = 'reconstruction of'
              AND ftb.type = 'reconstruction of'
              AND fb.subject_identity <> fa.subject_identity
              AND fb.subject_identity NOT IN (
                  SELECT b.idoriginal_poem
                  FROM data.original_poem_verse a
                  JOIN data.original_poem_verse b ON a.idgroup = b.idgroup
                  WHERE a.idoriginal_poem = ANY(%s)
                    AND b.idoriginal_poem <> a.idoriginal_poem
                  GROUP BY a.idoriginal_poem, b.idoriginal_poem
              )
        """, (occurrence_ids_int, occurrence_ids_int))
        type_rows = pg_cursor.fetchall()

        # 4️⃣ Combine all rows
        all_rows = verse_rows + type_rows
        print(f"Total related occurrences to insert: {len(all_rows)}")

        # 5️⃣ Bulk insert into SQLite
        if all_rows:
            cursor.executemany("""
                INSERT OR IGNORE INTO occurrence_related_occurrences
                (occurrence_id, related_occurrence_id, relation_definition_id)
                VALUES (?, ?, ?)
            """, all_rows)
        else:
            print("No related occurrences found.")

    cursor.execute("COMMIT")
    print("Occurrence relations migrated successfully.")

    pg_cursor.close()
    pg_conn.close()

    # Mapping table updates
    mapping_table = f"mapping.mapping_{occ_index}"
    cursor.execute("BEGIN")
    cursor.execute(f"SELECT field_name FROM {mapping_table}")
    for (f,) in cursor.fetchall():
        if f in migrated:
            cursor.execute(f"UPDATE {mapping_table} SET mapped_to='migrated' WHERE field_name=?",(f,))
        elif f in skipped or f.startswith(("manuscript.","manuscript_content","contributor.","transcriber.","scribe.","person_subject.","genre.","metre.", "scribe_public.","scribe_public","transcriber_public","transcriber_public.*","acknowledgement.name.keyword","contributor_public.*","contributor_public")):
            cursor.execute(f"UPDATE {mapping_table} SET mapped_to='skipped' WHERE field_name=?",(f,))
    cursor.execute("COMMIT")

conn.close()
print("Occurrences, roles, person links, genres, metres, managements migrated; mapping table updated successfully.")


Fetched 1000 occurrences from ES...
Fetched 2000 occurrences from ES...
Fetched 3000 occurrences from ES...
Fetched 4000 occurrences from ES...
Fetched 5000 occurrences from ES...
Fetched 6000 occurrences from ES...
Fetched 7000 occurrences from ES...
Fetched 8000 occurrences from ES...
Fetched 9000 occurrences from ES...
Fetched 10000 occurrences from ES...
Fetched 11000 occurrences from ES...
Fetched 12000 occurrences from ES...
Fetched 13000 occurrences from ES...
Total occurrences fetched: 13017
Committed 1000 occurrences...
Committed 2000 occurrences...
Committed 3000 occurrences...
Committed 4000 occurrences...
Committed 5000 occurrences...
Committed 6000 occurrences...
Committed 7000 occurrences...
Committed 8000 occurrences...
Committed 9000 occurrences...
Committed 10000 occurrences...
Committed 11000 occurrences...
Committed 12000 occurrences...
Committed 13000 occurrences...
Total occurrences processed and committed: 13017
Found 13017 occurrences. Fetching related occurrence

## Manuscripts

In [22]:
import sqlite3
from elasticsearch import Elasticsearch

es = Elasticsearch("http://localhost:19200")

main_db_path = "export_data.sqlite"
mapping_db_path = "export_data_mapping.sqlite"

conn = sqlite3.connect(main_db_path, timeout=60, isolation_level=None)
cursor = conn.cursor()
cursor.execute("PRAGMA journal_mode = WAL;")
cursor.execute("PRAGMA busy_timeout = 60000;")
cursor.execute("PRAGMA foreign_keys = ON;")
cursor.execute(f"ATTACH DATABASE '{mapping_db_path}' AS mapping;")

# Ensure the manuscript_person_roles table exists
cursor.execute("""
CREATE TABLE IF NOT EXISTS manuscript_person_roles (
    manuscript_id TEXT NOT NULL,
    person_id TEXT NOT NULL,
    role_id TEXT NOT NULL,
    PRIMARY KEY (manuscript_id, person_id, role_id),
    FOREIGN KEY (manuscript_id) REFERENCES manuscripts(id),
    FOREIGN KEY (person_id) REFERENCES persons(id),
    FOREIGN KEY (role_id) REFERENCES roles(id)
)
""")

# New table for manuscript -> management links
cursor.execute("""
CREATE TABLE IF NOT EXISTS manuscript_management (
    manuscript_id TEXT NOT NULL,
    management_id TEXT NOT NULL,
    PRIMARY KEY (manuscript_id, management_id),
    FOREIGN KEY (manuscript_id) REFERENCES manuscripts(id),
    FOREIGN KEY (management_id) REFERENCES management(id)
)
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS acknowledgements (
    id TEXT PRIMARY KEY,
    name TEXT NOT NULL
)
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS manuscript_acknowledgement (
    manuscript_id TEXT NOT NULL,
    acknowledgement_id TEXT NOT NULL,
    PRIMARY KEY (manuscript_id, acknowledgement_id),
    FOREIGN KEY (manuscript_id) REFERENCES manuscripts(id),
    FOREIGN KEY (acknowledgement_id) REFERENCES acknowledgements(id)
)
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS content (
    id TEXT PRIMARY KEY,
    name TEXT NOT NULL
)
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS manuscript_content (
    manuscript_id TEXT NOT NULL,
    content_id TEXT NOT NULL,
    PRIMARY KEY (manuscript_id, content_id),
    FOREIGN KEY (manuscript_id) REFERENCES manuscripts(id),
    FOREIGN KEY (content_id) REFERENCES content(id)
)
""")


cursor.execute("""
CREATE TABLE IF NOT EXISTS manuscript_identification (
    manuscript_id TEXT NOT NULL,
    identification_id TEXT NOT NULL,
    PRIMARY KEY (manuscript_id, identification_id),
    FOREIGN KEY (manuscript_id) REFERENCES manuscripts(id),
    FOREIGN KEY (identification_id) REFERENCES identifications(id)
)
""")


# Map ES fields to roles in the roles table
role_field_to_role_name = {
    "illuminator": "Illuminator",
    "contributor": "Contributor",
    "scribe": "Scribe",
    "patron": "Patron",
    "owner": "Owner",
    "creator": "Creator",
    "person_content": "Content"
}

# Get manuscript index
indices = es.cat.indices(format="json")
indices = [i["index"] for i in indices if i["index"].startswith("dbbe_dev")]
manuscript_index = next((i for i in indices if i.endswith("manuscripts")), None)

def scroll_all(index):
    r = es.search(index=index, body={"query": {"match_all": {}}, "size": 1000}, scroll="2m")
    sid = r["_scroll_id"]
    hits = r["hits"]["hits"]
    out = hits[:]
    while hits:
        r = es.scroll(scroll_id=sid, scroll="2m")
        hits = r["hits"]["hits"]
        out.extend(hits)
    return out

def ensure_column(table, column, coltype):
    cursor.execute(f"PRAGMA table_info({table})")
    cols = {row[1] for row in cursor.fetchall()}
    if column not in cols:
        cursor.execute(f"ALTER TABLE {table} ADD COLUMN {column} {coltype}")


if manuscript_index:
    hits = scroll_all(manuscript_index)
    cursor.execute("BEGIN")

    for h in hits:
        s = h["_source"]
        mid = str(s.get("id", h["_id"]))

        for col, typ in [
            ("name", "TEXT"),
            ("date_floor_year", "INTEGER"),
            ("date_ceiling_year", "INTEGER"),
            ("created", "TEXT"),
            ("modified", "TEXT"),
            ("number_of_occurrences", "INTEGER"),
        ]:
            ensure_column("manuscripts", col, typ)


        cursor.execute("""
        INSERT INTO manuscripts (
            id,
            name,
            date_floor_year,
            date_ceiling_year,
            created,
            modified,
            number_of_occurrences
        )
        VALUES (?, ?, ?, ?, ?, ?, ?)
        ON CONFLICT(id) DO UPDATE SET
            name = excluded.name,
            date_floor_year = excluded.date_floor_year,
            date_ceiling_year = excluded.date_ceiling_year,
            created = excluded.created,
            modified = excluded.modified,
            number_of_occurrences = excluded.number_of_occurrences
        """, (
            mid,
            s.get("name"),
            s.get("date_floor_year"),
            s.get("date_ceiling_year"),
            s.get("created"),
            s.get("modified"),
            s.get("number_of_occurrences"),
        ))


        # Roles
        def get_role_id(role_name):
            cursor.execute("SELECT id FROM roles WHERE LOWER(name)=LOWER(?)", (role_name,))
            row = cursor.fetchone()
            return row[0] if row else None

        for role_field, role_name_in_table in role_field_to_role_name.items():
            role_id = get_role_id(role_name_in_table)
            if not role_id:
                continue
            persons = s.get(role_field, [])
            if isinstance(persons, list):
                for p in persons:
                    person_id = str(p.get("id", ""))
                    if not person_id:
                        continue
                    cursor.execute("SELECT 1 FROM persons WHERE id=?", (person_id,))
                    if cursor.fetchone():
                        cursor.execute(
                            "INSERT OR IGNORE INTO manuscript_person_roles (manuscript_id, person_id, role_id) VALUES (?, ?, ?)",
                            (mid, person_id, role_id)
                        )

        # Managements
        for mgmt in s.get("management", []):
            mgmt_id = str(mgmt.get("id",""))
            mgmt_name = mgmt.get("name","")
            if mgmt_id and mgmt_name:
                cursor.execute("INSERT OR IGNORE INTO management (id, name) VALUES (?,?)", (mgmt_id, mgmt_name))
                cursor.execute("INSERT OR IGNORE INTO manuscript_management (manuscript_id, management_id) VALUES (?,?)", (mid, mgmt_id))
        for content in s.get("content", []):
            content_id = str(content.get("id",""))
            content_name = content.get("name","")
            if content_id and content_name:
                cursor.execute("INSERT OR IGNORE INTO content (id, name) VALUES (?,?)", (content_id, content_name))
                cursor.execute("INSERT OR IGNORE INTO manuscript_content (manuscript_id, content_id) VALUES (?,?)", (mid, content_id))
        for ack in s.get("acknowledgement", []):
            ack_id = str(ack.get("id", ""))
            ack_name = ack.get("name", "")
            if ack_id and ack_name:
                cursor.execute(
                    "INSERT OR IGNORE INTO acknowledgements (id, name) VALUES (?, ?)",
                    (ack_id, ack_name)
                )
                cursor.execute(
                    "INSERT OR IGNORE INTO manuscript_acknowledgement (manuscript_id, acknowledgement_id) VALUES (?, ?)",
                    (mid, ack_id)
                )



        MANUSCRIPT_IDENT_TYPE_MAP = {
            "diktyon": "diktyon",
        }

        for es_field, ident_type in MANUSCRIPT_IDENT_TYPE_MAP.items():
            for identifier in s.get(es_field, []):
                if not identifier:
                    continue

                ident_id = str(uuid.uuid4())  # new UUID for this identification

                # Insert into identifications table
                cursor.execute(
                    """
                    INSERT OR IGNORE INTO identifications
                    (id, type, identifier_value)
                    VALUES (?, ?, ?)
                    """,
                    (ident_id, ident_type, identifier)
                )

                # Link to manuscript
                cursor.execute(
                    """
                    INSERT OR IGNORE INTO manuscript_identification
                    (manuscript_id, identification_id)
                    VALUES (?, ?)
                    """,
                    (mid, ident_id)
                )


    cursor.execute("COMMIT")

    mapping_table = f"mapping.mapping_{manuscript_index}"
    cursor.execute("BEGIN")
    cursor.execute(f"SELECT field_name FROM {mapping_table}")

    for (f,) in cursor.fetchall():
        if f == "manuscript.id":
            cursor.execute(f"UPDATE {mapping_table} SET mapped_to='migrated' WHERE field_name=?",(f,))
        elif f in ["acknowledgement","contributor.id",
                   "illuminator.id","scribe.id","patron.id","creator.id",
                   "owner.id","person_content.id","diktyon","management",
                   "created","modified","date_floor_year","date_ceiling_year","id","number_of_occurrences","name","content.name",
            "content.id"]:\
            cursor.execute(f"UPDATE {mapping_table} SET mapped_to='migrated' WHERE field_name=?",(f,))
        elif f in [
            "acknowledgement.name.keyword",
                "contributor_public.id",
                "contributor_public.id_name",
                "contributor_public.id_name.keyword",
                "contributor_public.name",
                "contributor_public.name.keyword",
                "scribe",
                "scribe_public.id",
                "scribe_public.id_name.keyword",
                "scribe_public.id_name",
                "scribe_public.name.keyword",
                "scribe_public.name",
                "scribe_public",
                "illuminator",
                "illuminator.id",
                "illuminator.id_name",
                "illuminator.id_name.keyword",
                "illuminator.name",
                "illuminator.name.keyword",
                "illuminator_public",
                "illuminator_public.id",
                "illuminator_public.id_name",
                "illuminator_public.id_name.keyword",
                "illuminator_public.name",
                "illuminator_public.name.keyword",
                "patron",
                "patron.id",
                "patron.id_name",
                "patron.id_name.keyword",
                "patron.name",
                "patron.name.keyword",
                "patron_public",
                "patron_public.id",
                "patron_public.id_name",
                "patron_public.id_name.keyword",
                "patron_public.name",
                "patron_public.name.keyword",
                "creator",
                "creator.id",
                "creator.id_name",
                "creator.id_name.keyword",
                "creator.name",
                "creator.name.keyword",
                "creator_public",
                "creator_public.id",
                "creator_public.id_name",
                "creator_public.id_name.keyword",
                "creator_public.name",
                "creator_public.name.keyword",
                "owner",
                "owner.id",
                "owner.id_name",
                "owner.id_name.keyword",
                "owner.name",
                "owner.name.keyword",
                "owner_public",
                "owner_public.id",
                "owner_public.id_name",
                "owner_public.id_name.keyword",
                "owner_public.name",
                "owner_public.name.keyword",
                "person_content",
                "person_content.id",
                "person_content.id_name",
                "person_content.id_name.keyword",
                "person_content.name",
                "person_content.name.keyword",
                "person_content_public",
                "person_content_public.id",
                "person_content_public.id_name",
                "person_content_public.id_name.keyword",
                "person_content_public.name",
                "person_content_public.name.keyword",
                "scribe.id_name.keyword",
                "scribe.id_name",
                "scribe.name.keyword",
                "scribe.name",
                "contributor.id_name.keyword",
                "contributor.id_name",
                "contributor.name.keyword",
                "contributor.name",
                "diktyon.keyword",
            "management.name.keyword",
            "name.keyword",
            "diktyon_available"
            "content.display",
            "content.id_name.keyword",
            "content.id_name",
            "content.name.keyword",
            "content",

            ]:
            cursor.execute(f"UPDATE {mapping_table} SET mapped_to='skipped' WHERE field_name=?",(f,))
        elif f in ["management.id","management.name"]:
            cursor.execute(f"UPDATE {mapping_table} SET mapped_to='migrated' WHERE field_name=?",(f,))
        elif f in ["management.id_name","management.id_name.keyword"]:
            cursor.execute(f"UPDATE {mapping_table} SET mapped_to='skipped' WHERE field_name=?",(f,))
        elif f in ["acknowledgement.id","acknowledgement.name"]:
            cursor.execute(f"UPDATE {mapping_table} SET mapped_to='migrated' WHERE field_name=?", (f,))
        elif f in ["acknowledgement.id_name","acknowledgement.id_name.keyword"]:
            cursor.execute(f"UPDATE {mapping_table} SET mapped_to='skipped' WHERE field_name=?", (f,))

    cursor.execute("COMMIT")

conn.close()
print("Manuscript and person roles updated.")


Manuscript and person roles updated.


## Bibliographies

In [23]:
import sqlite3
from elasticsearch import Elasticsearch

es = Elasticsearch("http://localhost:19200")

main_db_path = "export_data.sqlite"
mapping_db_path = "export_data_mapping.sqlite"

conn = sqlite3.connect(main_db_path, timeout=60, isolation_level=None)
cursor = conn.cursor()
cursor.execute("PRAGMA journal_mode = WAL;")
cursor.execute("PRAGMA busy_timeout = 60000;")
cursor.execute("PRAGMA foreign_keys = ON;")
cursor.execute(f"ATTACH DATABASE '{mapping_db_path}' AS mapping;")

# 1️⃣ Ensure bibliography_person_roles exists
cursor.execute("""
CREATE TABLE IF NOT EXISTS bibliography_person_roles (
    bibliography_id TEXT NOT NULL,
    person_id TEXT NOT NULL,
    role_id TEXT NOT NULL,
    PRIMARY KEY (bibliography_id, person_id, role_id),
    FOREIGN KEY (bibliography_id) REFERENCES bibliographies(id),
    FOREIGN KEY (person_id) REFERENCES persons(id),
    FOREIGN KEY (role_id) REFERENCES roles(id)
)
""")

# 1b️⃣ Ensure biblio_category table exists
cursor.execute("""
CREATE TABLE IF NOT EXISTS biblio_category (
    id TEXT PRIMARY KEY,
    name TEXT NOT NULL
)
""")

# 1c️⃣ Create bibliography_management table
cursor.execute("""
CREATE TABLE IF NOT EXISTS bibliography_management (
    bibliography_id TEXT NOT NULL,
    management_id TEXT NOT NULL,
    PRIMARY KEY (bibliography_id, management_id),
    FOREIGN KEY (bibliography_id) REFERENCES bibliographies(id),
    FOREIGN KEY (management_id) REFERENCES management(id)
)
""")

# Add missing columns to bibliographies
cursor.execute("PRAGMA table_info(bibliographies)")
columns = [c[1] for c in cursor.fetchall()]
if "category_id" not in columns:
    cursor.execute("ALTER TABLE bibliographies ADD COLUMN category_id TEXT REFERENCES biblio_category(id)")
if "title" not in columns:
    cursor.execute("ALTER TABLE bibliographies ADD COLUMN title TEXT")
if "title_sort_key" not in columns:
    cursor.execute("ALTER TABLE bibliographies ADD COLUMN title_sort_key TEXT")

# 2️⃣ Map ES fields to roles
role_field_to_role_name = {
    "author": "Author",
    "editor": "Editor",
    "translator": "Translator",
    "supervisor": "Supervisor"
}

# 3️⃣ Get bibliography index
indices = es.cat.indices(format="json")
indices = [i["index"] for i in indices if i["index"].startswith("dbbe_dev")]
bibliography_index = next((i for i in indices if i.endswith("bibliographies")), None)

if bibliography_index:

    # Scroll helper
    def scroll_all(index):
        r = es.search(index=index, body={"query": {"match_all": {}}, "size": 500}, scroll="2m")
        scroll_id = r["_scroll_id"]
        hits = r["hits"]["hits"]
        all_hits = hits[:]
        while True:
            if not hits:
                break
            r = es.scroll(scroll_id=scroll_id, scroll="2m")
            scroll_id = r["_scroll_id"]
            hits = r["hits"]["hits"]
            if not hits:
                break
            all_hits.extend(hits)
        return all_hits

    hits = scroll_all(bibliography_index)
    print(f"Fetched {len(hits)} bibliographies from Elasticsearch")

    # 4️⃣ Insert bibliographies, categories, titles
    cursor.execute("BEGIN")
    for h in hits:
        s = h["_source"]
        bid = str(s.get("id", h["_id"]))
        cursor.execute("INSERT OR IGNORE INTO bibliographies (id) VALUES (?)", (bid,))

        # Category
        type_info = s.get("type", {})
        type_id = str(type_info.get("id", ""))
        type_name = type_info.get("name", "")
        if type_id and type_name:
            cursor.execute("INSERT OR IGNORE INTO biblio_category (id, name) VALUES (?, ?)", (type_id, type_name))
            cursor.execute("UPDATE bibliographies SET category_id=? WHERE id=?", (type_id, bid))

        # Titles
        title = s.get("title", None)
        title_sort_key = s.get("title_sort_key", None)
        if title or title_sort_key:
            cursor.execute("UPDATE bibliographies SET title=?, title_sort_key=? WHERE id=?", (title, title_sort_key, bid))

    cursor.execute("COMMIT")

    # 5️⃣ Insert bibliography_person_roles
    cursor.execute("BEGIN")
    for h in hits:
        s = h["_source"]
        bid = str(s.get("id", h["_id"]))

        def get_role_id(role_name):
            cursor.execute("SELECT id FROM roles WHERE LOWER(name)=LOWER(?)", (role_name,))
            row = cursor.fetchone()
            return row[0] if row else None

        for role_field, role_name_in_table in role_field_to_role_name.items():
            role_id = get_role_id(role_name_in_table)
            if not role_id:
                continue
            persons = s.get(role_field, [])
            if isinstance(persons, list):
                for p in persons:
                    person_id = str(p.get("id",""))
                    if not person_id:
                        continue
                    cursor.execute("SELECT 1 FROM persons WHERE id=?", (person_id,))
                    person_exists = cursor.fetchone()
                    cursor.execute("SELECT 1 FROM bibliographies WHERE id=?", (bid,))
                    bibliography_exists = cursor.fetchone()
                    if person_exists and bibliography_exists:
                        cursor.execute(
                            "INSERT OR IGNORE INTO bibliography_person_roles (bibliography_id, person_id, role_id) VALUES (?,?,?)",
                            (bid, person_id, role_id)
                        )
    cursor.execute("COMMIT")

    # 6️⃣ Insert bibliography managements
    cursor.execute("BEGIN")
    for h in hits:
        s = h["_source"]
        bid = str(s.get("id", h["_id"]))
        for mgmt in s.get("management", []):
            mgmt_id = str(mgmt.get("id",""))
            mgmt_name = mgmt.get("name","")
            if mgmt_id and mgmt_name:
                cursor.execute("INSERT OR IGNORE INTO management (id,name) VALUES (?,?)", (mgmt_id, mgmt_name))
                cursor.execute("INSERT OR IGNORE INTO bibliography_management (bibliography_id, management_id) VALUES (?,?)", (bid, mgmt_id))
    cursor.execute("COMMIT")

    # 7️⃣ Update mapping table
    mapping_table = f"mapping.mapping_{bibliography_index}"
    cursor.execute("BEGIN")
    cursor.execute(f"SELECT field_name FROM {mapping_table}")

    skip_fields = [
        "supervisor.id_name.keyword","supervisor.id_name","supervisor.name.keyword","supervisor.name",
        "translator.id_name.keyword","translator.id_name","translator.name.keyword","translator.name",
        "author.id_name.keyword","author.id_name","author.name.keyword","author.name",
        "editor.id_name.keyword","editor.id_name","editor.name.keyword","editor.name"
        "supervisor.id_name.keyword", "supervisor.id_name", "supervisor.name.keyword", "supervisor.name", "supervisor_public.id", "supervisor_public.id_name.keyword", "supervisor_public.id_name", "supervisor_public.name.keyword", "supervisor_public.name", "test.id", "test.id_name.keyword", "test.id_name", "test.name.keyword", "test.name", "test", "test_public.id", "test_public.id_name.keyword", "test_public.id_name", "test_public.name.keyword", "test_public.name", "translator.id_name.keyword", "translator.id_name", "translator.name.keyword", "translator.name", "translator", "translator_public.id", "translator_public.id_name.keyword", "translator_public.id_name", "translator_public.name.keyword", "translator_public.name", "translator_public", "type.id_name.keyword", "type.id_name", "type.name.keyword", "author.id_name.keyword", "author.id_name", "author.name.keyword", "author.name", "author", "author_last_name.keyword", "author_last_name", "author_last_name_public.keyword", "author_last_name_public", "author_public.id", "author_public.id_name.keyword", "author_public.id_name", "author_public.name.keyword", "author_public.name", "author_public", "editor.id_name.keyword", "editor.id_name", "editor.name.keyword", "editor.name", "editor", "editor_public.id", "editor_public.id_name.keyword", "editor_public.id_name", "editor_public.name.keyword", "editor_public.name", "editor_public"
    ]

    for (f,) in cursor.fetchall():
        if f == "bibliography.id":
            cursor.execute(f"UPDATE {mapping_table} SET mapped_to='migrated' WHERE field_name=?", (f,))
        elif f in ["author.id","editor.id","translator.id","supervisor.id"]:
            cursor.execute(f"UPDATE {mapping_table} SET mapped_to='migrated' WHERE field_name=?", (f,))
        elif f in ["type.id","type.name","title","title_sort_key"]:
            cursor.execute(f"UPDATE {mapping_table} SET mapped_to='migrated' WHERE field_name=?", (f,))
        elif f in ["management.id","management.name"]:
            cursor.execute(f"UPDATE {mapping_table} SET mapped_to='migrated' WHERE field_name=?", (f,))
        elif f in ["management.id_name","management.id_name.keyword"] or f in skip_fields:
            cursor.execute(f"UPDATE {mapping_table} SET mapped_to='skipped' WHERE field_name=?", (f,))
    cursor.execute("COMMIT")

conn.close()
print("Bibliographies, person roles, categories, managements, and titles updated successfully.")


Fetched 3206 bibliographies from Elasticsearch
Bibliographies, person roles, categories, managements, and titles updated successfully.


## Export kyriaki

### Occurrences

In [24]:
import sqlite3
import pandas as pd
from pathlib import Path

# Connect to the correct database
DB_PATH = Path("export_data.sqlite").resolve()
conn = sqlite3.connect(DB_PATH)

query = """
WITH verse_data AS (
    SELECT
        o.id AS occurrence_id,
        COUNT(v.id) AS number_of_verses,
        GROUP_CONCAT(v.text, ' ') AS text
    FROM occurrences o
    LEFT JOIN verses v
        ON v.occurrence_id = o.id
    GROUP BY o.id
),

genre_data AS (
    SELECT
        og.occurrence_id,
        GROUP_CONCAT(g.name, ';') AS genres
    FROM occurrence_genres og
    JOIN genres g ON g.id = og.genre_id
    GROUP BY og.occurrence_id
),

metre_data AS (
    SELECT
        om.occurrence_id,
        GROUP_CONCAT(m.name, ';') AS metres
    FROM occurrence_metres om
    JOIN metres m ON m.id = om.metre_id
    GROUP BY om.occurrence_id
),

person_data AS (
    SELECT
        opr.occurrence_id,
        GROUP_CONCAT(
            p.full_name || ' [' || r.id || ': ' || r.name || ']',
            ';'
        ) AS persons
    FROM occurrence_person_roles opr
    JOIN persons p ON p.id = opr.person_id
    JOIN roles r ON r.id = opr.role_id
    WHERE p.is_historical_person = 1
    GROUP BY opr.occurrence_id
),

text_status_data AS (
    SELECT
        ots.occurrence_id,
        GROUP_CONCAT(ts.name, ';') AS text_status
    FROM occurrence_text_statuses ots
    JOIN text_statuses ts ON ts.id = ots.text_status_id
    GROUP BY ots.occurrence_id
),

type_data AS (
    SELECT
        tocc.occurrence_id,
        GROUP_CONCAT(t.id, ';') AS types
    FROM type_occurrences tocc
    JOIN types t ON t.id = tocc.type_id
    GROUP BY tocc.occurrence_id
),

related_occurrence_data AS (
    SELECT
        oro.occurrence_id,
        GROUP_CONCAT(oro.related_occurrence_id, ';') AS related_occurrences
    FROM occurrence_related_occurrences oro
    GROUP BY oro.occurrence_id
)

SELECT
    o.id AS occurrence_id,
    vd.text,
    gd.genres,
    vd.number_of_verses,
    md.metres,
    pd.persons,
    tsd.text_status,
    o.public_comment,
    o.private_comment,
    o.title,
    o.palaeographical_info,
    o.contextual_info,
    o.date_floor_year,
    o.date_ceiling_year,
    o.location,
    o.manuscript_id,
    td.types,
    rod.related_occurrences
FROM occurrences o
LEFT JOIN verse_data vd ON vd.occurrence_id = o.id
LEFT JOIN genre_data gd ON gd.occurrence_id = o.id
LEFT JOIN metre_data md ON md.occurrence_id = o.id
LEFT JOIN person_data pd ON pd.occurrence_id = o.id
LEFT JOIN text_status_data tsd ON tsd.occurrence_id = o.id
LEFT JOIN type_data td ON td.occurrence_id = o.id
LEFT JOIN related_occurrence_data rod ON rod.occurrence_id = o.id
"""

df = pd.read_sql_query(query, conn)
df.to_csv('occurrences.tsv', sep='\t', index=False)
df


,occurrence_id,text,genres,number_of_verses,metres,persons,text_status,public_comment,private_comment,title,palaeographical_info,contextual_info,date_floor_year,date_ceiling_year,location,manuscript_id,types,related_occurrences
0,24767,ζῆτα δ’ ἂρ’ ἀνδρομάχης καὶ ἕκτορο(ς) ἔστ’ ὀαρι...,Text-related epigram,1,Dactylic hexameter,Homer [1001: Subject],Text completely known,Mioni (1985: 245): 'Unicuique libro argumenta ...,Transcribed by RR in the Biblioteca Marciana.,,Written in red ink.,The epigram precedes Book 6 of the Iliad.\nIt ...,1301,1400,f. I: 70r,15777,6491,22469;22518;22573;22631;22660;22712;22887;2324...
1,18673,Θ(εο)ῦ τὸ δῶρον καὶ πόνος Ἰωάσαφ,Scribe-related epigram,1,Dodecasyllable,Ioasaph [4: Scribe],Text completely known,Lampros (1895: 310) dates the manuscript to th...,Over scribent: zie ms. (DN)\r\n\r\nTekst zo (z...,,,,1301,1400,,13827,3805,17042;17213;17336;17584;17624;17906;17958;1800...
2,20412,Λόγος πενθικὸς τῷ πενθεῖν βουλομένῳ.,Text-related epigram,1,Dodecasyllable,None,Text completely known,,,,,,1401,1500,,14781,4215,20414
3,24768,ἦτα δ' αἴας πολέμιζε μόνω μόνος ἕκτορι δίω:-,Text-related epigram,1,Dactylic hexameter,Homer [1001: Subject],Text completely known,Mioni (1985: 245): 'Unicuique libro argumenta ...,Mioni gives no text.\nTranscribed by RR in the...,,Written in red ink.,The epigram precedes Book 7 of the Iliad.,1301,1400,f. I: 80v,15777,6493,22470;22531;22576;22632;22662;22713;22888;2324...
4,20658,None,None,0,None,None,Text completely unknown,Sajdak (1914: 267): 'Epigrammata haec publicav...,"Sajdak (1914: 266-267): ""Epigrammata haec publ...",,,,1401,1500,,13930,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13012,37082,※ ἐξεσίη δ’ ἀχιλῆος ἀπειθέος ἐστὶν ἰῶτα:-,Text-related epigram,1,Dactylic hexameter,Dionysios Sinates [4: Scribe],Text completely known,,Created and transcribed by ASR.,,Written in red ink.,The verse precedes Book 9 of the Iliad.\nIt is...,1451,1475,f. 86r,14274,6475,22472;22578;22634;22664;22716;22890;23248;2337...
13013,37083,κάππα δ’ ἄρ ἀμφοτέρ(ων) σκοπιαζέμεν ἤλυθον ἄνδ...,Text-related epigram,1,Dactylic hexameter,Dionysios Sinates [4: Scribe],Text completely known,,Created and transcribed by ASR.,,Written in red ink.,The verse precedes Book 10 of the Iliad.,1451,1475,f. 97r,14274,6496,20801;22473;22536;22579;22635;22665;22717;2289...
13014,37084,+ λάβδα δ’ ἀριστῆας δανα(ῶν) ἔβάλον ἕκτορος ἄν...,Text-related epigram,1,Dactylic hexameter,Dionysios Sinates [4: Scribe],Text completely known,,Created and transcribed by ASR.,,Written in red ink.,The verse precedes Book 11 of the Iliad.,1451,1475,f. 106r,14274,6498,22476;22538;22580;22636;22667;22718;22892;2325...
13015,37135,Πόσος πόθος μοι σῶν καλῶν συγγραμμάτων. ᾿Ἐνεστ...,Author-related epigram;Text-related epigram,6,Dodecasyllable,Eusebios of Caesarea [1001: Subject];Origen o...,Text completely known,,,,,It follows Origenes' De Principiis.,1501,1600,f. 13v,37134,37136,None


### Types

In [25]:
import sqlite3
import pandas as pd

# Connect to the database
conn = sqlite3.connect('export_data.sqlite')

query = """
WITH cte_genres AS (
    SELECT
        t.id,
        GROUP_CONCAT(DISTINCT g.name) as genres
    FROM types t
    LEFT JOIN type_occurrences t_occ ON t.id = t_occ.type_id
    LEFT JOIN occurrence_genres og ON t_occ.occurrence_id = og.occurrence_id
    LEFT JOIN genres g ON og.genre_id = g.id
    GROUP BY t.id
),
cte_verses AS (
    SELECT
        t.id,
        COUNT(DISTINCT v.id) as number_of_verses
    FROM types t
    LEFT JOIN type_occurrences t_occ ON t.id = t_occ.type_id
    LEFT JOIN verses v ON t_occ.occurrence_id = v.occurrence_id
    GROUP BY t.id
),
cte_metres AS (
    SELECT
        t.id,
        GROUP_CONCAT(DISTINCT m.name) as metres
    FROM types t
    LEFT JOIN type_occurrences t_occ ON t.id = t_occ.type_id
    LEFT JOIN occurrence_metres om ON t_occ.occurrence_id = om.occurrence_id
    LEFT JOIN metres m ON om.metre_id = m.id
    GROUP BY t.id
),
cte_subjects AS (
    SELECT
        t.id,
        GROUP_CONCAT(DISTINCT s.name) as subjects
    FROM types t
    LEFT JOIN type_subject ts ON t.id = ts.type_id
    LEFT JOIN subjects s ON ts.subject_id = s.id
    GROUP BY t.id
),
cte_persons AS (
    SELECT
        t.id,
        GROUP_CONCAT(DISTINCT p.full_name) as persons
    FROM types t
    LEFT JOIN type_person_roles tpr ON t.id = tpr.type_id
    LEFT JOIN persons p ON tpr.person_id = p.id
    WHERE p.is_historical_person = 1
    GROUP BY t.id
),
cte_tags AS (
    SELECT
        t.id,
        GROUP_CONCAT(DISTINCT tag.name) as tags
    FROM types t
    LEFT JOIN type_tags tt ON t.id = tt.type_id
    LEFT JOIN tags tag ON tt.tag_id = tag.id
    GROUP BY t.id
),
cte_text_statuses AS (
    SELECT
        t.id,
        GROUP_CONCAT(DISTINCT ts_name.name) as text_status
    FROM types t
    LEFT JOIN type_text_statuses tts ON t.id = tts.type_id
    LEFT JOIN text_statuses ts_name ON tts.text_status_id = ts_name.id
    GROUP BY t.id
),
cte_occurrences_list AS (
    SELECT
        t.id,
        GROUP_CONCAT(DISTINCT t_occ.occurrence_id) as occurrences
    FROM types t
    LEFT JOIN type_occurrences t_occ ON t.id = t_occ.type_id
    GROUP BY t.id
),


cte_editorial_statuses AS (
    SELECT
        t.id,
        GROUP_CONCAT(DISTINCT es.name) AS editorial_status
    FROM types t
    LEFT JOIN type_editorial_status tes ON t.id = tes.type_id
    LEFT JOIN editorial_statuses es ON tes.editorial_status_id = es.id
    GROUP BY t.id
),

cte_related_types AS (
    SELECT
        trt.type_id as id,
        GROUP_CONCAT(DISTINCT trt.related_type_id) as related_types
    FROM type_related_types trt
    GROUP BY trt.type_id
)


SELECT
    t.id as type_id,
    t.text_original as text,
    t.public_comment,
    t.private_comment,
    t.title,
    tg.genres,
    COALESCE(tv.number_of_verses, 0) as number_of_verses,
    tm.metres,
    ts.subjects,
    tp.persons,
    ttags.tags,
    tts.text_status,
    te.editorial_status,
    tol.occurrences,
    rt.related_types
FROM types t
LEFT JOIN cte_genres tg ON t.id = tg.id
LEFT JOIN cte_verses tv ON t.id = tv.id
LEFT JOIN cte_metres tm ON t.id = tm.id
LEFT JOIN cte_subjects ts ON t.id = ts.id
LEFT JOIN cte_persons tp ON t.id = tp.id
LEFT JOIN cte_tags ttags ON t.id = ttags.id
LEFT JOIN cte_editorial_statuses te ON t.id = te.id
LEFT JOIN cte_text_statuses tts ON t.id = tts.id
LEFT JOIN cte_occurrences_list tol ON t.id = tol.id
LEFT JOIN cte_related_types rt ON t.id = rt.id
ORDER BY t.id
"""

df = pd.read_sql_query(query, conn)
conn.close()

# Replace commas with semicolons in columns with multiple values
columns_to_replace = ['genres', 'metres', 'subjects', 'persons', 'tags', 'text_status', 'occurrences']
for col in columns_to_replace:
    df[col] = df[col].str.replace(',', ';')

df.to_csv('types.tsv', sep='\t', index=False)

# Display the dataframe
df

,type_id,text,public_comment,private_comment,title,genres,number_of_verses,metres,subjects,persons,tags,text_status,editorial_status,occurrences,related_types
0,1855,Αἶνος φλόγα σβέννυσι τῶν τριῶν νέων.,This metrical title precedes Ode 7. It belongs...,"Behoort tot Mearns' (1914, 13) ""other, much le...",,Text-related epigram,4,Dodecasyllable,None,None,Reference to content of the book;Title,Text completely known,Critical text,17238;17384;22694;27019,"1934,1939,1986,1994,2090,2092,2155,3200,3202,3204"
1,1857,"Αἶνος Θεῷ, χάρις τε καὶ δόξα πρέπει,\nτῷ δόντι...",,"Corrected accentuation, also in http://www.dbb...",,Scribe-related epigram,6,Dodecasyllable,None,None,Completion of the book;Entreaty for first person,Text completely known,Not a critical text,16977;17385;17386,"1869,2121"
2,1858,Εὐαγγελιστῶν τοὺς θεοπνεύστους λόγους\nτόμοις ...,"There is another type with the same incipit, b...","Comm. Klaas:\n\nVoornamelijk de ""materiële"" kw...",,Author-related epigram,13,Dodecasyllable,Four Evangelists,None,Beauty of the book;Address to the reader;Entre...,Text completely known,Critical text,17013,None
3,1860,"Ἱδρώτων οὐκ ὀλίγων Χριστέ, Θεέ μου,\nἀδίκως στ...",Unique epigram (see occurrence for details abo...,Vastgestelde tekst is eerste kritische tekst (...,,Patron-related epigram,8,Dodecasyllable,Jesus Christ (10 BC-1 BC - 30-33);Ioannes (105...,Jesus Christ;Ioannes,Dedication;Reference to content of the book;En...,Text completely known,Critical text,17060,None
4,1862,"Ἀγγελικῶν γραφίδων ἀμαρύγματα πολλὰ κιχήσας,\n...",Even though this epigram and type 1863 (cf. re...,Why not related type with http://www.dbbe.ugen...,Ἐπίγραμμα εἰς τὸν μακάριον Διονύσιον,Author-related epigram;Text-related epigram,96,Dactylic hexameter,Pseudo-Dionysios the Areopagite (5th c. - 6th c.),Pseudo-Dionysios the Areopagite,Spiritual value of the book;Praise of the author,Text completely known,Critical text,16936;17346;17381;17946;17950;17954;19165;1962...,1863
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5279,7417,Οὐρανίαν με καλοῦσι θνητοί τ’ ἀθάνατοι τε·\nοὔ...,The poem forms part of a cycle on the nine Muses.,Created by JBe.\nType revised by ASR (18/09/20...,,Text-related epigram,2,Elegiacs,Nine Muses,Theodore Gazes,Title,Text completely known,Critical text,27076,"7410,7411,7412,7413,7414,7415,7416,7418"
5280,7418,Καλλιόπη πέλομαι Διὸς εὐειδέστατον ἔρνος·\nφθέ...,The poem forms part of a cycle on the nine Muses.,Created by JBe.\nType revised by ASR (18/09/20...,,Text-related epigram,2,Elegiacs,Nine Muses;Zeus,Theodore Gazes,Title,Text completely known,Critical text,27077,"7410,7411,7412,7413,7414,7415,7416,7417"
5281,7419,Τέλος σὺν [Θ]εῷ τῆσδε τῆς βίβλου ταύτης.,,Not in Vassis (RR). Partly not readable.,,Scribe-related epigram,1,Dodecasyllable,None,None,Completion of the book,Text completely known,Not a critical text,27142,None
5282,7420,"Δόξα σοι πάντων ἕνεκα παντοκράτωρ,\nτῷ συντελε...",,Created by JBe.,,Scribe-related epigram,2,Dactylic hexameter,None,None,Completion of the book,Text completely known,Not a critical text,27143,None


### manuscripts

In [26]:
import sqlite3
import pandas as pd

# Connect to DB
db_path = "export_data.sqlite"
conn = sqlite3.connect(db_path)
conn.row_factory = sqlite3.Row
cursor = conn.cursor()

# Aggregate manuscript info with historical persons only
sql = """
WITH manuscript_contents AS (
    SELECT
        mc.manuscript_id,
        GROUP_CONCAT(c.name, '; ') AS content
    FROM manuscript_content mc
    JOIN content c ON mc.content_id = c.id
    GROUP BY mc.manuscript_id
),
manuscript_persons AS (
    SELECT
        mpr.manuscript_id,
        GROUP_CONCAT(p.full_name || ' [' || r.name || ']', '; ') AS persons
    FROM manuscript_person_roles mpr
    JOIN persons p ON mpr.person_id = p.id
    JOIN roles r ON mpr.role_id = r.id
    WHERE p.is_historical_person = 1
    GROUP BY mpr.manuscript_id
),
manuscript_dates AS (
    SELECT
        id AS manuscript_id,
        date_floor_year ||
        CASE
            WHEN date_ceiling_year IS NOT NULL THEN '-' || date_ceiling_year
            ELSE ''
        END AS date_range
    FROM manuscripts
)
SELECT
    m.id AS manuscript_id,
    m.name AS manuscript_name,
    mc.content,
    mp.persons,
    md.date_range AS date
FROM manuscripts m
LEFT JOIN manuscript_contents mc ON mc.manuscript_id = m.id
LEFT JOIN manuscript_persons mp ON mp.manuscript_id = m.id
LEFT JOIN manuscript_dates md ON md.manuscript_id = m.id
ORDER BY m.id
"""

# Load into pandas DataFrame
df_manuscripts = pd.read_sql_query(sql, conn)
conn.close()
df_manuscripts.to_csv('manuscripts.tsv', sep='\t', index=False)
# Show the DataFrame
df_manuscripts.head()


,manuscript_id,manuscript_name,content,persons,date
0,12756,SINAI - Mone tes Hagias Aikaterines - gr. 213,Biblica > Novum Testamentum > Evangeliarium; B...,None,967-967
1,12757,OXFORD - Christ Church College 28,Biblica > Novum Testamentum > Evangeliarium; B...,Gregorios [Scribe],1351-1400
2,12758,OXFORD - Christ Church College 29,Biblica > Novum Testamentum > Evangeliarium; B...,Konon [Scribe]; Chariton [Scribe]; Hilarion [P...,1131-1131
3,12759,OXFORD - Bodleian Library - Clarke 15,Biblica > Vetus Testamentum > Psalterium; Bibl...,Markos [Patron],1078-1078
4,12760,WASHINGTON D.C. - Dumbarton Oaks 3,Biblica > Vetus Testamentum > Psalterium; Bibl...,None,1084-1084
